# Students
- GHAITH Sarahnour (M2QF & ENSIIE)
- ROISEUX Thomas (M2QF & ENSIIE)

# Introduction
## Context

The goal of this project is to study 
the temporal evolution of temperature and wind in France, across one year.

## Required packages
- `pandas` : to manipulate dataframes.
- `numpy` : to manipulate arrays.
- `matplotlib` : to plot graphs.
- `cartopy` : to plot maps.
- `IPython` : to display dataframes in Jupyter Notebook.
- `scikit-learn` : to use machine learning algorithms.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import warnings
from datetime import datetime, timedelta
from typing import Dict

from IPython.display import display

from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.decomposition import PCA
from functools import partial
from scipy.stats import norm
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

warnings.filterwarnings("ignore")

scatter = partial(plt.scatter, transform=ccrs.PlateCarree(), cmap=plt.colormaps()[7])

%matplotlib inline

## Data importation
### Preparing GPS dataframe

In [ ]:
gps_df = pd.read_csv("data/dataGPS.csv", header=None, sep=";")
gps_df.columns = ["ID", "Lattitude", "Longitude"]
gps_df["ID"] = gps_df["ID"].str.replace("TEMP", "")
gps_df.set_index("ID", inplace=True)

display(gps_df.head())

Now, we plot the GPS data from the file `dataGPS.csv` on a map of France.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

for x, y in zip(gps_df["Lattitude"], gps_df["Longitude"]):
    ax.plot(x, y, "r+")
plt.show()

### Preparing temperature dataframe and wind dataframe

In [ ]:
year = 2019
hours = [datetime(year, 1, 1, 0, 0, 0) + timedelta(hours=i) for i in range(8760)]

In [ ]:
temp_df = pd.read_csv("data/dataTemp.csv", header=None, sep=";", index_col=0)
temp_df.index.name = "ID"
for key in temp_df.index:
    temp_df.rename(index={key: key.replace("TEMP", "")}, inplace=True)
temp_df.columns = hours


display(temp_df.head())


wind_df = pd.read_csv("data/dataWind.csv", header=None, sep=";", index_col=0)
wind_df.index.name = "ID"
for key in wind_df.index:
    wind_df.rename(index={key: key.replace("VVENT", "")}, inplace=True)
wind_df.columns = hours


display(wind_df.head())

## Example: weather in Paris
We are goinng to study the weather in Paris, the capital of France, as an example.
It is located at 48.51° N, 2.21° E.

Let's firstplace in on a map.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

x, y = 2.217999, 48.512381
ax.plot(x, y, "r*", markersize=15)
ax.text(x, y, "Paris")
plt.show()

Now, we are going to plot the evolution of temperature and wind in Paris, across one year.

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("Temperature in Paris")
plt.plot(temp_df.columns, temp_df.iloc[33, :], color="blue")
plt.show()

In [ ]:
gmm = GaussianMixture(n_components=4, covariance_type="full", random_state=0)
gmm.fit(temp_df.iloc[33, :].values.reshape(-1, 1))
ll = gmm.score_samples(temp_df.iloc[33, :].values.reshape(-1, 1))

ordered_values = np.argsort(temp_df.iloc[33, :])

plt.figure(figsize=(20, 10))
plt.title("Temperature in Paris")
plt.hist(temp_df.iloc[33, :], bins=50, color="blue", density=True, label="Temperature")
plt.plot(
    [temp_df.iloc[33, i] for i in ordered_values],
    [np.exp(ll)[i] for i in ordered_values],
    color="red",
    label="Gaussian Mixture Model",
)
plt.axis(xmin=np.min(temp_df.iloc[33, :]), xmax=np.max(temp_df.iloc[33, :]), ymin=0)
plt.legend()
plt.show()

For this model, we used 4 components for the Gaussian Mixture as we have 4 seasons in a year.

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("Wind in Paris")
plt.plot(wind_df.columns, wind_df.iloc[33, :], color="blue")
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("Wind in Paris")
plt.hist(wind_df.iloc[33, :], color="blue", label="Wind", density=True)
plt.plot(
    np.linspace(0, np.max(wind_df.iloc[33, :]), 100),
    norm.pdf(
        np.linspace(0, np.max(wind_df.iloc[33, :]), 100),
        np.mean(wind_df.iloc[33, :]),
        np.std(wind_df.iloc[33, :]),
    ),
    color="red",
    label="Normal distribution associated",
)
plt.axis(xmin=0, xmax=np.max(wind_df.iloc[33, :]))
plt.legend()
plt.show()

# Preliminaries
## Cities selection
We are going to select 3 more cities in France, to study the weather in different regions.

We chose:
- Strasbourg (48.58° N, 7.75° E);
- Nice (43.70° N, 7.26° E);
- Brest (48.39° N, 4.48° W);

In [ ]:
def find_closest_point(x: float, y: float, df: pd.DataFrame) -> Dict[str, float | str]:
    """Get the closest point to the given coordinates in the given dataframe.

    Args:
        x (float): longitude
        y (float): lattitude
        df (pd.DataFrame): dataframe with columns Longitude and Lattitude

    Returns:
        dict[str, float | str]: closest point
    """
    distances = np.sqrt((df["Longitude"] - x) ** 2 + (df["Lattitude"] - y) ** 2)
    dic = df.iloc[np.argmin(distances)].to_dict()
    dic["ID"] = df.index[np.argmin(distances)]
    return dic


strasbourg = find_closest_point(48.5734053, 7.7521113, gps_df)
print("Strasbourg:", (strasbourg["Longitude"], strasbourg["Lattitude"]))
nice = find_closest_point(43.7009358, 7.2683912, gps_df)
print("Nice:", (nice["Longitude"], nice["Lattitude"]))
brest = find_closest_point(48.390528, -4.486008, gps_df)
print("Brest:", (brest["Longitude"], brest["Lattitude"]))

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

x, y = 2.217999, 48.512381
ax.plot(x, y, "r*", markersize=15)
ax.text(x, y, "Paris")
ax.plot(strasbourg["Lattitude"], strasbourg["Longitude"], "r*", markersize=15)
ax.text(strasbourg["Lattitude"], strasbourg["Longitude"], "Strasbourg")
ax.plot(nice["Lattitude"], nice["Longitude"], "r*", markersize=15)
ax.text(nice["Lattitude"], nice["Longitude"], "Nice")
ax.plot(brest["Lattitude"], brest["Longitude"], "r*", markersize=15)
ax.text(brest["Lattitude"], brest["Longitude"], "Brest")
plt.show()

We are now going to plot the evolution of temperature and wind in these cities, across one year.

In [ ]:
plt.figure(figsize=(20, 10))
plt.title(f"Temperature")
plt.xlabel("Time")
plt.ylabel("Temperature")
for dict, names in zip((strasbourg, nice, brest), ("Strasbourg", "Nice", "Brest")):
    wind_id = dict["ID"]
    temp_id = dict["ID"]
    plt.plot(temp_df.columns, temp_df.loc[temp_id, :], label=names, alpha=0.5)

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.title(f"Wind")
plt.xlabel("Time")
plt.ylabel("Wind speed")
for dict, names in zip((strasbourg, nice, brest), ("Strasbourg", "Nice", "Brest")):
    wind_id = dict["ID"]
    temp_id = dict["ID"]
    plt.plot(wind_df.columns, wind_df.loc[wind_id, :], label=names, alpha=0.5)

plt.legend()
plt.show()

## Clustering
We are going to cluster the cities in France, using the temperature and wind data.
We will use 2 different clustering algorithms:
- $K$-means;
- hierarchical clustering.
### $K$-means
We will use $K=4$ for the number of clusters.

In [ ]:
k_means_wind, k_means_temp = KMeans(n_clusters=4), KMeans(n_clusters=4)
k_means_wind.fit(wind_df)
k_means_temp.fit(temp_df)

classifications = pd.DataFrame(
    k_means_wind.predict(wind_df), index=wind_df.index, columns=["K Wind"]
)
classifications["K Temperature"] = k_means_temp.predict(temp_df)

display(classifications.head())

We are going to plot the clusters on a map, first for the wind.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["K Wind"],
)
plt.show()

Now we are going to do the same for the temperature.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["K Temperature"],
)
plt.show()

We here notice that the clusters are not exactly the same but are quite close enough.
### Hierarchical clustering
After using the $K$-means algorithm, we are going to use hierarchical clustering, with the same number of clusters.

In [ ]:
agg_wind, agg_temp = AgglomerativeClustering(n_clusters=4), AgglomerativeClustering(
    n_clusters=4
)
agg_wind = agg_wind.fit(wind_df)
agg_temp = agg_temp.fit(temp_df)

classifications["A Wind"] = agg_wind.labels_
classifications["A Temperature"] = agg_temp.labels_

display(classifications.head())

We are going to plot the clusters on a map, first for the wind.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["A Wind"],
)
plt.show()

And now for the temperature.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["A Temperature"],
)
plt.show()

## Average clustering
We are going to compute the average of the temperature and wind data, for each city, then classify the cities using the average data.
### $K$-means

In [ ]:
averages = (wind_df + temp_df) / 2

In [ ]:
k_means = KMeans(n_clusters=4)
k_means.fit(wind_df)

classifications["K Average"] = k_means.predict(averages)

display(classifications.head())

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["K Average"],
)
plt.show()

### Hierarchical clustering

In [ ]:
k_means = AgglomerativeClustering(n_clusters=4)
k_means.fit(wind_df)

classifications["A Average"] = k_means.labels_

display(classifications.head())

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()
scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["A Average"],
)
plt.show()

## Conclusion
Fortunately, this algorithm is considered as "naive", as computing the average between wind and temperature is not relevant, and so is the resuling classification.
# Wind clustering
We are now going to cluster only wind data, but we will use new algorithms.
## Raw data
Using the raw time series, this was done in the previous part, under the title "Clustering".
Here is the resulting plot as a reminder.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["K Wind"],
)
plt.show()

We therefore notice that the wind categories are not very relevant, and seems related to the different climates in France.
## Principal component analysis
We will now perform a principal component analysis on the wind data, to reduce the dimension of the data.
We have $n=259$ cities and $p=8460$ time stamps.
Our goal is to reduce the dimension of the data, while keeping the most relevant information.
Each time stamp is separated by only one hour, so we can assume that the data is highly correlated.
Keeping all the time stamps would be redundant, so we are going to reduce the dimension of the data.

In [ ]:
corr_matrix = wind_df.corr()
corr_matrix_diff = corr_matrix.diff(axis=1).dropna(axis=1)

As we have features that describes the wind hour per hour, we will evaluate the differences of the correlation between the features, as it will give a good idea about the correlatoin between two consecutive time stamps.

In [ ]:
print("Correlation matrix differences mean:", corr_matrix_diff.mean().mean())
print("Correlation matrix differences std:", corr_matrix_diff.std().std())
print("Correlation matrix differences min:", corr_matrix_diff.abs().min().min())
print("Correlation matrix differences max:", corr_matrix_diff.abs().max().max())
print("Correlation matrix differences median:", corr_matrix_diff.median().median())

The statistical information of the correlation matrix is given just above: this shows that the features are highly correlated, as the mean of the differences along the columns is close to $0$.
Before doing the PCA, we will do another PCA with all the components, to determine how many ones we must keep in order to keep $95\,\%$ of the variance.

## Computing the best number of components

In [ ]:
pca = PCA(n_components=200)

pca.fit(wind_df)

variance_ratio = pca.explained_variance_ratio_

cumulative_variance_ratio = np.cumsum(variance_ratio)

plt.figure(figsize=(20, 10))
plt.title("Variance ratio")
plt.plot(cumulative_variance_ratio)
plt.xlabel("Principal components number")
plt.ylabel("Amount of explained variance")
plt.show()

print(
    "Minimum number of components to explain 90% of the variance:",
    np.argmax(cumulative_variance_ratio > 0.90) + 1,
)
print(
    "Minimum number of components to explain 95% of the variance:",
    np.argmax(cumulative_variance_ratio > 0.95) + 1,
)
print(
    "Minimum number of components to explain 99% of the variance:",
    np.argmax(cumulative_variance_ratio > 0.99) + 1,
)

As we want to keep at least $95\,\%$ of the variance, we will keep the first $k=38$ principal components.
As reminder, we had $p=8460$ features, so we reduced the dimension of the data by a factor of $223$.

## PCA and clustering
We are now going to do the PCA with $k=10$ components, and then cluster the data using $K$-means and hierarchical clustering.
### PCA

In [ ]:
pca = PCA(n_components=10)
pca.fit(wind_df)

reduced_wind_df = pd.DataFrame(pca.transform(wind_df), index=wind_df.index)
display(reduced_wind_df.head())

Let's analyse this new dataframe.

In [ ]:
reduced_corr_matrix = reduced_wind_df.corr()
display(reduced_corr_matrix)
print("Reduced correlation matrix mean:", reduced_corr_matrix.mean().mean())
print("Reduced correlation matrix std:", reduced_corr_matrix.std().std())
print("Reduced correlation matrix min:", reduced_corr_matrix.abs().min().min())
print("Reduced correlation matrix max:", reduced_corr_matrix.abs().max().max())
print("Reduced correlation matrix median:", reduced_corr_matrix.median().median())

The previous results shows that all features are now decorrelated, as the mean of the differences along the columns is close to $0$.

### $K$-means
We will use $K=4$ for the number of clusters.

We are going to do the same classification as before but now, using the PCA data.

In [ ]:
reduced_k_means_wind = KMeans(n_clusters=4)
reduced_k_means_wind.fit(reduced_wind_df)

classifications["PCA-K Wind"] = reduced_k_means_wind.predict(reduced_wind_df)

display(classifications.head())

We will now plot the clusters on a map.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["PCA-K Wind"],
)

plt.show()

We have now a better classification of the wind in France, as the clusters are more homogeneous.
Let's try to do the same with hierarchical clustering.
### Hierarchical clustering

In [ ]:
reduced_ac = AgglomerativeClustering(n_clusters=4)
reduced_ac.fit(reduced_wind_df)

classifications["PCA-A Wind"] = reduced_ac.labels_

display(classifications.head())

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["PCA-A Wind"],
)

plt.show()

This seems to be a better classification than the previous one, as the clusters are more homogeneous are even more correlated with the different climate zones in France.
## Conclusion
The last model seems to be the closest one to the reality, as the clusters are more homogeneous and more correlated with the different climate zones in France.
This confirms that the PCA was a good way to reduce the dimension of the data, while keeping the most relevant information.
# Temperature clustering
This part is similar to the previous one, but we will use temperature data instead of wind data.
## Raw data
Using the raw time series, this was done in the part I, under the title "Clustering".
Here are the resulting plots as a reminder.
### $K$-means

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["K Temperature"],
)

plt.show()

### Hierarchical clustering

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["A Temperature"],
)

plt.show()

### Conclusion
These models don't seem very accurate, as close cities along the Mediterrean coast are in different clusters.
## Feature engineering
Here, we are going to perform a functional principal component analysis on the temperature data, to reduce the dimension of the data.
Before doing it, let's write a few words about this method.

The fPCA is a generalization of the PCA, as it is used to analyse data that are functions.
It is used to analyse the variability of the data, and to reduce the dimension of the data.

The fPCA is based on the Karhunen-Loève expansion, which is a generalization of the Fourier series.
To perform this analysis, we will use the [`fdasrsf`](https://fdasrsf-python.readthedocs.io/en/latest/fpca_example.html) package, which is a Python package that implements the fPCA.
This package needs a number of components to perform the analysis: we are therefore going to compute the best number of components to keep, in order to keep $95\,\%$ of the variance.

fPCA can be done by 3 different ways:
- vertical fPCA, which applies on the time;
- horizontal fPCA, which applies on the subjects;
- joint fPCA, which applies on both.

In our case, as we want to reduce the dimension of the data, we will use the vertical fPCA.

To determine the best number of components to keep, we will use the elbow method.
Fortuneately, the `fdasrsf` package has a function that does it for us: we just have to supply the argument `var_exp=0.95` o the function `calc_fpca` and it will return the fPCA with the best number of components to keep $95\,\%$ of the variance.

However, due to package restrictions (quite hard to install on Windows) and high computing time (even using an Azure Linux VM with 8 cores and 16 GB of RAM, optimized for computation), after 9 hours of computing, only 2 components were computed.

We then will do a regular PCA to reduce the dimension of the data, and so we are going to apply the same process as before.
Let's first see the correlation between contiguous timestamps.

In [ ]:
corr_matrix = temp_df.corr()
corr_matrix_diff = corr_matrix.diff(axis=1).dropna(axis=1)

print("Correlation matrix differences mean:", corr_matrix_diff.mean().mean())
print("Correlation matrix differences std:", corr_matrix_diff.std().std())
print("Correlation matrix differences min:", corr_matrix_diff.abs().min().min())
print("Correlation matrix differences max:", corr_matrix_diff.abs().max().max())
print("Correlation matrix differences median:", corr_matrix_diff.median().median())

Regarding this results, we will apply the same process as before to determine the optimal number of components to keep.

In [ ]:
pca = PCA(n_components=200)

pca.fit(wind_df)

variance_ratio = pca.explained_variance_ratio_

cumulative_variance_ratio = np.cumsum(variance_ratio)

plt.figure(figsize=(20, 10))
plt.title("Variance ratio")
plt.plot(cumulative_variance_ratio)
plt.xlabel("Principal components number")
plt.ylabel("Amount of explained variance")
plt.show()

print(
    "Minimum number of components to explain 90% of the variance:",
    np.argmax(cumulative_variance_ratio > 0.90) + 1,
)
print(
    "Minimum number of components to explain 95% of the variance:",
    np.argmax(cumulative_variance_ratio > 0.95) + 1,
)
print(
    "Minimum number of components to explain 99% of the variance:",
    np.argmax(cumulative_variance_ratio > 0.99) + 1,
)

Here are the results depend on the amount of variance we want to keep.

We are now going to keep only the 10 first components, as it is the same number as the one we kept for the wind data.

In [ ]:
pca = PCA(n_components=10)
pca.fit(wind_df)

reduced_temp_df = pd.DataFrame(pca.transform(temp_df), index=temp_df.index)
display(reduced_temp_df.head())

Let's analyse the results.

In [ ]:
reduced_corr_matrix = reduced_temp_df.corr()
display(reduced_corr_matrix)
print("Reduced correlation matrix mean:", reduced_corr_matrix.mean().mean())
print("Reduced correlation matrix std:", reduced_corr_matrix.std().std())
print("Reduced correlation matrix min:", reduced_corr_matrix.abs().min().min())
print("Reduced correlation matrix max:", reduced_corr_matrix.abs().max().max())
print("Reduced correlation matrix median:", reduced_corr_matrix.median().median())

We notice that all features are now decorrelated, as the mean of the differences along the columns is close to $0$.
## Model based clustering
We are now going to cluster the data using the model based clustering.
We will use the Gaussian Mixture model.

To do so, we are going to study four different assumptions for the gaussian mixture density:
- spherical with equal volume;
- spherical with unequal volume;
- diagonal, varying volume and shape;
- independency between features, varying volume and shape.

Our goal is to find the best number of components to use, for each assumption.
### Spherical with unequal volume
We will use the Bayesian Information Criterion (BIC) to determine the best number of components to use.
We will use the `GaussianMixture` class from the `sklearn.mixture` package, which implements the Gaussian Mixture model for a number of components $K$ between 1 and 10 and we will retain the one that minimizes the BIC.

The argument `covariance_type` is used to specify the covariance matrix of the gaussian mixture.
The value `spherical` means that each component has its own single variance.

In [ ]:
bic = []

for k in range(2, 30):
    gaussian_spherical_equal = GaussianMixture(
        n_components=k, covariance_type="spherical", random_state=0
    )

    gaussian_spherical_equal.fit(reduced_temp_df)

    bic.append(gaussian_spherical_equal.bic(reduced_temp_df))

print("Best number of clusters:", np.argmin(bic) + 2)

plt.figure(figsize=(20, 10))
plt.title("BIC")
plt.plot(range(2, 30), bic)
plt.xlabel("Number of clusters")
plt.ylabel("BIC")
plt.show()

Using this number of components (28), we will now do our model based clustering.

In [ ]:
gaussian_spherical_equal = GaussianMixture(
    n_components=np.argmin(bic) + 2, covariance_type="spherical", random_state=0
)

labels = gaussian_spherical_equal.fit_predict(reduced_temp_df)

classifications["GMM - Spherical - Unequal"] = labels

Now, we can plot them on a map.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["GMM - Spherical - Unequal"],
)

plt.show()

Acccording to the BIC, we should use $K=28$ components.
This leads to a very good classification, as the clusters are very homogeneous and are correlated with the different climate zones in France.
However, this number of components is very high, and it might not be very relevant to use it.

Let's try to do the same with the other assumptions.

### Spherical with equal volume
We will do the same as before, but now with the assumption that all components have the same volume.
This is done by using the argument `covariance_type='tied'` in the `GaussianMixture` class.

In [ ]:
bic = []

for k in range(2, 30):
    gaussian_spherical_equal = GaussianMixture(
        n_components=k, covariance_type="tied", random_state=0
    )

    gaussian_spherical_equal.fit(reduced_temp_df)

    bic.append(gaussian_spherical_equal.bic(reduced_temp_df))

print("Best number of clusters:", np.argmin(bic) + 2)

plt.figure(figsize=(20, 10))
plt.title("BIC")
plt.plot(range(2, 30), bic)
plt.xlabel("Number of clusters")
plt.ylabel("BIC")
plt.show()

gaussian_spherical_equal = GaussianMixture(
    n_components=np.argmin(bic) + 2, covariance_type="tied", random_state=0
)

labels = gaussian_spherical_equal.fit_predict(reduced_temp_df)

classifications["GMM - Spherical - Equal"] = labels

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["GMM - Spherical - Equal"],
)

plt.show()

20 clusters provide the best BIC.
This is also lower than the previous one, but still very high.

## Diagonal, varying volume and shape
We will do the same as before, but now with the assumption that the covariance matrix is diagonal, and that the volume and shape of the components can vary.
This is done by using the argument `covariance_type='diag'` in the `GaussianMixture` class.

In [ ]:
bic = []

for k in range(2, 30):
    gaussian_diag = GaussianMixture(
        n_components=k, covariance_type="diag", random_state=0
    )

    gaussian_diag.fit(reduced_temp_df)

    bic.append(gaussian_diag.bic(reduced_temp_df))

print("Best number of clusters:", np.argmin(bic) + 2)

plt.figure(figsize=(20, 10))
plt.title("BIC")
plt.plot(range(2, 30), bic)
plt.xlabel("Number of clusters")
plt.ylabel("BIC")
plt.show()

gaussian_spherical_equal = GaussianMixture(
    n_components=np.argmin(bic) + 2, covariance_type="diag", random_state=0
)

labels = gaussian_spherical_equal.fit_predict(reduced_temp_df)

classifications["GMM - Diagonal"] = labels

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["GMM - Diagonal"],
)

plt.show()

Having only $14$ clusters provides a better classification than the previous one, as the clusters are more homogeneous and are correlated with the different climate zones in France.

## Independency between features, varying volume and shape
To achieve that, we will use the argument `covariance_type='full'` in the `GaussianMixture` class.

In [ ]:
bic = []

for k in range(2, 30):
    gaussian_diag = GaussianMixture(
        n_components=k, covariance_type="full", random_state=0
    )

    gaussian_diag.fit(reduced_temp_df)

    bic.append(gaussian_diag.bic(reduced_temp_df))

print("Best number of clusters:", np.argmin(bic) + 2)

plt.figure(figsize=(20, 10))
plt.title("BIC")
plt.plot(range(2, 30), bic)
plt.xlabel("Number of clusters")
plt.ylabel("BIC")
plt.show()

gaussian_spherical_equal = GaussianMixture(
    n_components=np.argmin(bic) + 2, covariance_type="full", random_state=0
)

labels = gaussian_spherical_equal.fit_predict(reduced_temp_df)

classifications["GMM - Full"] = labels

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["GMM - Full"],
)

plt.show()

This time, we have more clusters, however the classification seems as good as the previous one, because all clusters are homogeneous and are correlated with the different climate zones in France.
Also, we know that temperature and winds are continuous variables, so it is more relevant to use this assumption.
### Conclusion
Among all assumptions, the last two ones seem to be the best ones, as it provides a good classification, with a reasonable number of clusters, and with homogeneous ones that are correlated with the different climate zones in France.
## Spectral clustering
We are now going to cluster the data using spectral clustering.
This method is based on the graph theory, and is used to cluster data that are not linearly separable.
We will do this clustering on the fPCA data.

In [ ]:
scores = []

for k in range(4, 30):
    sp_clustering = SpectralClustering(n_clusters=k, n_components=10)
    labels = sp_clustering.fit_predict(reduced_temp_df)

    silhouette_avg = silhouette_score(reduced_temp_df, labels)

    scores.append(silhouette_avg)

print("Best number of clusters:", np.argmax(scores) + 4)

sp_clustering = SpectralClustering(n_clusters=np.argmax(scores) + 4, n_components=10)
sp_clustering.fit(reduced_temp_df)


classifications["FPCA-Spectral Temperature"] = sp_clustering.labels_

display(classifications.head())

We can now plot the spectral clustering results on a map.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    gps_df["Lattitude"],
    gps_df["Longitude"],
    c=classifications["FPCA-Spectral Temperature"],
)

plt.show()

# Clustering wind and temperature together
We are now going to cluster the wind and temperature data together, using the raw time series.
## Approach
First, we are going to make a dataframe of all wind and temperature data, for each city.

We are going to try two different clustering methods:
1. The first one will use a PCA then a $K$-means clustering.
2. The second one will be made of the same PCA (fPCA cannot be used for the same reasons as before) followed by a spectral clustering.
## Data preparation

In [ ]:
aggregated_df = gps_df.join(wind_df, how="inner").join(
    temp_df, how="inner", lsuffix="_wind", rsuffix="_temp"
)

if aggregated_df.shape[0] != gps_df.shape[0]:
    raise RuntimeError("Some rows were lost during the join operation")

display(f"Aggregated DF shape: {aggregated_df.shape}")
display(aggregated_df.head(10))
display(aggregated_df.tail(10))

## First approach: PCA and $K$-means
### PCA
We are going to use the same way as before to determine the number of components to keep.

In [ ]:
pca = PCA(n_components=200)

pca.fit(aggregated_df.drop(columns=["Lattitude", "Longitude"]))

variance_ratio = pca.explained_variance_ratio_

cumulative_variance_ratio = np.cumsum(variance_ratio)

plt.figure(figsize=(20, 10))
plt.title("Variance ratio")
plt.plot(cumulative_variance_ratio)
plt.xlabel("Principal components number")
plt.ylabel("Amount of explained variance")
plt.show()

print(
    "Minimum number of components to explain 90% of the variance:",
    np.argmax(cumulative_variance_ratio > 0.90) + 1,
)
print(
    "Minimum number of components to explain 95% of the variance:",
    np.argmax(cumulative_variance_ratio > 0.95) + 1,
)
print(
    "Minimum number of components to explain 99% of the variance:",
    np.argmax(cumulative_variance_ratio > 0.99) + 1,
)

As we want to explain at least $95\,\%$ of the variance, we will keep the first $k=34$ principal components.

In [ ]:
pca = PCA(n_components=np.argmax(cumulative_variance_ratio > 0.95) + 1)

display(f"{pca=}")

reduced_aggregated_data = pd.DataFrame(
    pca.fit_transform(aggregated_df.drop(columns=["Lattitude", "Longitude"])),
    index=aggregated_df.index,
)

display(reduced_aggregated_data.head())

Using this new dataframe, we are going to cluster the data using $K$-means.
But this time, we will try to find the optimal number of clusters, using the elbow method.
This method consists in plotting the inertia of the model, which is the sum of the squared distances between each point and its closest centroid.
Then, we will choose the number of clusters where the inertia starts to decrease slowly or stops strongly decreasing.
### $K$-means

In [ ]:
k_values = range(4, 30)

inertia = []
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(reduced_aggregated_data)
    inertia.append(kmeans.inertia_)

inertia = np.array(inertia)
inertia_ratio = inertia[:-1] / inertia[1:]

plt.figure(figsize=(20, 10))
plt.plot(k_values, inertia, marker="o")
plt.xlabel("Number of clusters")
plt.ylabel("Inertia")
plt.title("Elbow method")
plt.show()

We are going to analyse the differences of the inertia, to determine the optimal number of clusters.

In [ ]:
inertia_ratio = inertia[:-1] / inertia[1:]
plt.figure(figsize=(20, 10))
plt.plot(k_values[1:], inertia_ratio, marker="o")
plt.xlabel("Number of clusters")
plt.ylabel("Inertia ratio")
plt.title("Elbow method")
plt.show()

Regarding the previous plot, we will choose $K=11$ for the number of clusters.
### Plotting the clusters on a map

In [ ]:
aggregated_classification = pd.DataFrame(gps_df)

k_means = KMeans(n_clusters=11)

k_means.fit(reduced_aggregated_data)
aggregated_classification["K-means"] = k_means.predict(reduced_aggregated_data)
display(aggregated_classification.head())

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    aggregated_classification["Lattitude"],
    aggregated_classification["Longitude"],
    c=aggregated_classification["K-means"],
)

plt.show()

We are now going to implement the second approach then compare the results.
## Second approach: PCA and spectral clustering
### PCA
We are going to keep the same PCA results as before.
### Spectral clustering
Before doing the spectral clustering, we will determine the optimal number of clusters.

This time, we are going to select the number of clusters that maximizes the silhouette score.

The silhouette score is a metric that measures how similar a point is to its own cluster, compared to other clusters.

In [ ]:
scores = []

for k in k_values:
    sp_clustering = SpectralClustering(
        n_clusters=k, n_components=np.argmax(cumulative_variance_ratio > 0.95) + 1
    )
    labels = sp_clustering.fit_predict(reduced_aggregated_data)

    silhouette_avg = silhouette_score(reduced_aggregated_data, labels)

    scores.append(silhouette_avg)

Now, we are going to plot the silhouette score.

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("Silhouette score")
plt.plot(k_values, scores, marker="o")
plt.xlabel("Number of clusters")
plt.ylabel("Silhouette score")
plt.show()

In [ ]:
print("The best number of clusters is:", np.argmax(scores) + 4)

Using this plot, we will choose the best number of clusters.

In [ ]:
sp_clustering = SpectralClustering(
    n_clusters=np.argmax(scores) + 4,
    n_components=np.argmax(cumulative_variance_ratio > 0.95) + 1,
)
sp_clustering.fit(reduced_aggregated_data)

aggregated_classification["Spectral"] = sp_clustering.labels_
display(aggregated_classification.head())

Let's count the number of cities in each cluster.

In [ ]:
display(aggregated_classification["Spectral"].value_counts().sort_index())

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

scatter(
    aggregated_classification["Lattitude"],
    aggregated_classification["Longitude"],
    c=aggregated_classification["Spectral"],
)

plt.show()

## Conclusion between the two approaches
At first sight, the first approach seems to be the best one, as the clusters are more homogeneous and seems to be related to the different climate zones in France.
Also, in the second one, we have silhouettes that are negative, which means that some points are closer to other clusters than their own so the classification is not very relevant.

The spectral clustering also seems to provide a map that is not quite correlated with the different climate zones in France, but correlated with somehow France geography (mountains, plains, ...).

To go further, we could imagine using a neural network to classify the data, as it is a very powerful tool for classification. This would however require a full implementation using external libraries such as `TensorFlow` or `PyTorch`, which is not the goal of this project.
Also, regarding the amount of data, we cannot be sure to have better results than the ones we already have.
To speed up the computation, we could use a GPU with `CUDA` to perform the computations, but this would require a lot of time to implement, and we are not sure to have better results than the ones we already have.